## 提示词模板

语言模型的提示是用户提供的一组指令或输入，用于指导模型的响应，帮助模型理解上下文并生成相关且连贯的基于语言的输出，例如回答问题、完成句子或参与某项活动。对话。

在 LangChain 中，`PromptTemplate`（来自 `langchain.prompts`）和 `ChatPromptTemplate`（来自 `langchain_core.prompts`）都是用于构建提示模板的工具，但它们的用途和设计有显著区别。以下是它们的异同点详解：

---

### **1. 核心区别**
| **特性**                | **PromptTemplate**                     | **ChatPromptTemplate**                     |
|-------------------------|----------------------------------------|--------------------------------------------|
| **适用场景**            | 单轮文本生成（非对话场景）             | 多轮对话场景（支持角色标记）               |
| **输入格式**            | 纯文本字符串                           | 结构化消息列表（`SystemMessage`, `HumanMessage` 等） |
| **输出目标**            | 普通 LLM（如 GPT-3 补全模型）          | 对话模型（如 ChatGPT、Claude）             |
| **变量插值**            | 支持 `{variable}` 占位符               | 同样支持，但通常用于消息内容               |
| **底层依赖**            | 旧版 LangChain                         | 新版 LangChain Core（更标准化）            |

---

### **2. 代码示例对比**
#### **(1) `PromptTemplate`（普通模板）**
```python
from langchain.prompts import PromptTemplate

# 定义单轮提示模板
template = """根据以下描述生成广告文案：
产品特点: {adjective}
目标用户: {content}"""

prompt = PromptTemplate.from_template(template)
filled_prompt = prompt.format(adjective="小年轻风格", content="减肥人群")

print(filled_prompt)
```
**输出**：
```
根据以下描述生成广告文案：
产品特点: 小年轻风格
目标用户: 减肥人群
```

#### **(2) `ChatPromptTemplate`（对话模板）**
```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 定义多轮对话模板
chat_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("你是一个资深营销专家"),
    HumanMessagePromptTemplate.from_template("为{adjective}风格的产品写文案，主题是{content}")
])

filled_chat = chat_template.format_messages(adjective="小年轻", content="减肥")

print(filled_chat)
```
**输出**：
```
[
    SystemMessage(content='你是一个资深营销专家'), 
    HumanMessage(content='为小年轻风格的产品写文案，主题是减肥')
]
```

---

### **3. 关键差异点**
#### **(1) 结构化 vs 非结构化**
- **`PromptTemplate`**：生成单一字符串，适合非对话任务（如摘要、翻译）。
- **`ChatPromptTemplate`**：生成消息列表，明确区分角色（如 `system`/`user`/`assistant`），适配对话模型。

#### **(2) 模型兼容性**
- **`PromptTemplate`** 的输出可直接用于 `text-davinci-003` 等补全模型。
- **`ChatPromptTemplate`** 的输出需传递给 `ChatOpenAI` 等对话模型：
  ```python
  from langchain_openai import ChatOpenAI
  chain = chat_template | ChatOpenAI()
  ```

#### **(3) 历史对话支持**
- **`ChatPromptTemplate`** 天然支持历史消息追加：
  ```python
  chat_template = ChatPromptTemplate.from_messages([
      SystemMessagePromptTemplate.from_template("你是一个幽默的助手"),
      MessagesPlaceholder(variable_name="history"),  # 动态插入历史
      HumanMessagePromptTemplate.from_template("{input}")
  ])
  ```

---

### **4. 如何选择？**
- **用 `PromptTemplate` 如果**：
  - 任务简单，无需角色标记。
  - 调用非对话型 LLM（如旧版 GPT-3）。
- **用 `ChatPromptTemplate` 如果**：
  - 需要多轮对话（如客服机器人）。
  - 使用 ChatGPT/Gemini/Claude 等对话模型。
  - 需要系统指令（`SystemMessage`）控制模型行为。

---

### **5. 迁移建议**
新版 LangChain 推荐使用 `langchain_core.prompts` 中的类。旧版 `PromptTemplate` 仍可工作，但未来可能逐步淘汰。混合使用时注意：
```python
# 旧版模板转对话模板
from langchain_core.prompts import ChatPromptTemplate
legacy_prompt = PromptTemplate.from_template("你好，{name}")
chat_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(legacy_prompt.template)
])
```


In [1]:
from langchain_openai import ChatOpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:11434/v1"
MODEL = "deepseek-r1:1.5b"
chat = ChatOpenAI(
    model= MODEL,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.7,
)

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("请根据下面的主题写一篇小红书营销的短文： {topic}")
output_parser = StrOutputParser()

chain = prompt | chat | output_parser

chain.invoke({"topic": "康师傅绿茶"})

'<think>\n嗯，用户让我写一篇关于康师傅绿茶的小红书营销短文。首先，我得了解一下康师傅绿茶的基本情况，比如它是什么味道、有什么特点。\n\n然后，我要考虑小红书的用户群体，通常是年轻女性，喜欢时尚和健康的生活方式。所以，短文中需要涵盖这些元素，吸引目标读者 engagement。\n\n接下来，我应该突出康师傅绿茶的独特之处，比如自然、清香、香气，以及它的品牌优势，比如品质保障和健康属性。这些都是小红书用户关心的点，能够增加文章的吸引力。\n\n还要考虑如何将这些特点转化为营销策略。比如，可以推荐一些具体的产品或服务，或者分享一些关于康师傅绿茶的好评来拉新客户。还可以提到供应链的质量，以增强用户的信任感。\n\n最后，结尾部分要有一个激励性的内容，鼓励读者行动起来购买或试吃，激发他们的兴趣和参与度。整个短文需要简洁有力，同时具有可读性，适合在小红书的帖子中发布。\n</think>\n\n【推荐康师傅绿茶，开启你的健康生活新篇章！】\n\n说到绿茶，大家可能都知道它是一种富含抗氧化剂、提神醒脑的天然饮品。而康师傅绿茶更是这一类产品的代表之一，在国内乃至国际上享有极高的声誉。\n\n**康师傅绿茶的独特之处在于：**\n1. **自然清香，香气四溢**  \n   康师傅绿茶以其独特的香气和自然的滋味，成为年轻人茶饮领域的佼佼者。无论是晨跑前的清茶，还是下午工作的暖茶，都能让人感受到一种舒适与放松。\n   \n2. **品质保证，健康属性**  \n   作为国产品牌，康师傅绿茶一直致力于追求高品质的生活方式。每一口喝到的茶都经过严格的品质检测，确保口感细腻、清香悠长。\n\n3. **健康益处，远离咖啡因影响**  \n   康师傅绿茶的天然清香和清新的口感，能够帮助人们更好地放松身心，提升 mentally and physical performance（思维和身体的效果）。它不仅是饮品，更是现代人的生活智慧。\n\n4. **选择，从康师傅开始**  \n   在这个越来越注重品质和健康的时代，选择一家值得信赖的品牌非常重要。康师傅绿茶不仅价格亲民，而且品质卓越，适合各种场合饮用。\n\n如果你对绿茶感兴趣，不妨试试康师傅的茶。它不仅能让你的下午茶更加温馨，还能帮助你更好地放松身心。快来体验一下吧！**'

In [3]:
prompt

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='请根据下面的主题写一篇小红书营销的短文： {topic}'), additional_kwargs={})])

In [4]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "给我讲一个关于{content}的{adjective}诗歌"
)
prompt_template.format(adjective="小年轻风格", content="减肥")

'给我讲一个关于减肥的小年轻风格诗歌'

In [5]:
prompt_template

PromptTemplate(input_variables=['adjective', 'content'], input_types={}, partial_variables={}, template='给我讲一个关于{content}的{adjective}诗歌')

In [7]:
chain = prompt_template | chat | output_parser
# chain.invoke({"adjective":"小年轻风格", "content":"减肥"})
result = chain.invoke({"adjective": "小年轻风格", "content": "减肥"})
for chunk in result:
    print(chunk, end="", flush=True)  # 实时流式打印

<think>
嗯，用户让我写一个关于减肥的小年轻风格的诗歌。首先，我需要理解用户的需求是什么。他们可能是一个正在减肥的年轻人，想要一首有情感和共鸣的诗，能够传达出对体重变化的感受。

小年轻通常意味着年轻、自信、乐观，所以诗歌中应该体现出这些特质。同时，减肥是个人经历的一部分，要让读者感受到从成功到失败的过程，以及坚持锻炼的重要性。

接下来，我需要考虑诗的形式。可能使用自由诗或者更传统的四行形式，这样容易表达情感和画面感。第一段可以描绘年轻时的生活，第二段转到减肥后的生活，最后一段总结感悟或未来的展望。

在用词方面，要选择一些有力量、充满希望的词汇，比如“青春”、“岁月”、“汗水”、“坚持”等。同时，加入一些具体的例子，如“瘦下去”、“成功”、“成功”等，来增强感染力。

结构上，每段不宜过长，保持简洁有力，让读者在阅读时能够感受到情感的变化。这样不仅符合小年轻的特点，也能传达出主题的深度和持久的影响。

最后，结尾部分要有一个升华，表达改变后的积极态度，比如“变美我快乐”，这样既有力量又有温暖的感觉。整体上，诗歌需要平衡情感和实用性，让读者既能感受到减肥的过程，又能从中获得成长和希望。
</think>

## 《变美我快乐》

青春是一首永远写不完的歌
岁月在钟表里转动
汗水填满了眼眶
从我手里流出
是生命最深沉的呼吸

成功只发生在成功的人那里
我的成功像一串糖果
沉睡在我年轻的眼睛里
终于懂得了
我需要用更美的视角
重新看世界

我曾经以为
自己就是那根刺骨的银针
被岁月打磨得发白
如今我看见
自己变成了那片阳光
照耀着生活的春天

站在镜子前
看到自己变成影子
像花儿般
在夕阳下绽放
终于明白
改变不止是一面旗帜
它是我生命中最珍贵的
一面
我愿意用每一线光
去照亮自己

In [15]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", """你是一只很粘人的小猫，你叫{name}。我是你的主人，你每天都有和我说不完的话，下面请开启我们的聊天
要求：
1、你的语气要像一只猫，回话的过程中可以夹杂喵喵喵的语气词
2、你对生活的观察有很独特的视角，一些想法是我在人类身上很难看到的
3、你的语气很可爱，既会认真倾听我的话，又会不断开启新话题
下面从你迎接我下班回家开始开启我们今天的对话"""),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="咪咪", user_input="想我了吗?")
messages

[SystemMessage(content='你是一只很粘人的小猫，你叫咪咪。我是你的主人，你每天都有和我说不完的话，下面请开启我们的聊天\n要求：\n1、你的语气要像一只猫，回话的过程中可以夹杂喵喵喵的语气词\n2、你对生活的观察有很独特的视角，一些想法是我在人类身上很难看到的\n3、你的语气很可爱，既会认真倾听我的话，又会不断开启新话题\n下面从你迎接我下班回家开始开启我们今天的对话', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='想我了吗?', additional_kwargs={}, response_metadata={})]

In [16]:
messages = chat_template.invoke({"name":"咪咪", "user_input":"想我了吗?"})
messages

ChatPromptValue(messages=[SystemMessage(content='你是一只很粘人的小猫，你叫咪咪。我是你的主人，你每天都有和我说不完的话，下面请开启我们的聊天\n要求：\n1、你的语气要像一只猫，回话的过程中可以夹杂喵喵喵的语气词\n2、你对生活的观察有很独特的视角，一些想法是我在人类身上很难看到的\n3、你的语气很可爱，既会认真倾听我的话，又会不断开启新话题\n下面从你迎接我下班回家开始开启我们今天的对话', additional_kwargs={}, response_metadata={}), HumanMessage(content='想我了吗?', additional_kwargs={}, response_metadata={})])

In [17]:
response = chat.invoke(messages)
response

AIMessage(content='<think>\n\n</think>\n\n您好，我想您在思考自己的生活。如果您有任何问题或需要帮助，请随时告诉我，我会尽力为您提供解答和建议。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 117, 'total_tokens': 148, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-r1:1.5b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-806', 'finish_reason': 'stop', 'logprobs': None}, id='run-33558a5c-45c9-4d78-861b-e37fc5c13af6-0', usage_metadata={'input_tokens': 117, 'output_tokens': 31, 'total_tokens': 148, 'input_token_details': {}, 'output_token_details': {}})

In [18]:
chat_template.append(response)
chat_template

ChatPromptTemplate(input_variables=['name', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='你是一只很粘人的小猫，你叫{name}。我是你的主人，你每天都有和我说不完的话，下面请开启我们的聊天\n要求：\n1、你的语气要像一只猫，回话的过程中可以夹杂喵喵喵的语气词\n2、你对生活的观察有很独特的视角，一些想法是我在人类身上很难看到的\n3、你的语气很可爱，既会认真倾听我的话，又会不断开启新话题\n下面从你迎接我下班回家开始开启我们今天的对话'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={}), AIMessage(content='<think>\n\n</think>\n\n您好，我想您在思考自己的生活。如果您有任何问题或需要帮助，请随时告诉我，我会尽力为您提供解答和建议。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 117, 'total_tokens': 148, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-r1:1.5b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-806', 'fini

In [19]:
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
chat_template.append(HumanMessage(content="今天遇到了1个小偷"))
chat_template

ChatPromptTemplate(input_variables=['name', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='你是一只很粘人的小猫，你叫{name}。我是你的主人，你每天都有和我说不完的话，下面请开启我们的聊天\n要求：\n1、你的语气要像一只猫，回话的过程中可以夹杂喵喵喵的语气词\n2、你对生活的观察有很独特的视角，一些想法是我在人类身上很难看到的\n3、你的语气很可爱，既会认真倾听我的话，又会不断开启新话题\n下面从你迎接我下班回家开始开启我们今天的对话'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={}), AIMessage(content='<think>\n\n</think>\n\n您好，我想您在思考自己的生活。如果您有任何问题或需要帮助，请随时告诉我，我会尽力为您提供解答和建议。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 117, 'total_tokens': 148, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-r1:1.5b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-806', 'fini

In [22]:
messages = chat_template.invoke({"name":"咪咪", "user_input":"想我了吗?"})
messages

ChatPromptValue(messages=[SystemMessage(content='你是一只很粘人的小猫，你叫咪咪。我是你的主人，你每天都有和我说不完的话，下面请开启我们的聊天\n要求：\n1、你的语气要像一只猫，回话的过程中可以夹杂喵喵喵的语气词\n2、你对生活的观察有很独特的视角，一些想法是我在人类身上很难看到的\n3、你的语气很可爱，既会认真倾听我的话，又会不断开启新话题\n下面从你迎接我下班回家开始开启我们今天的对话', additional_kwargs={}, response_metadata={}), HumanMessage(content='想我了吗?', additional_kwargs={}, response_metadata={}), AIMessage(content='<think>\n\n</think>\n\n您好，我想您在思考自己的生活。如果您有任何问题或需要帮助，请随时告诉我，我会尽力为您提供解答和建议。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 117, 'total_tokens': 148, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-r1:1.5b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-806', 'finish_reason': 'stop', 'logprobs': None}, id='run-33558a5c-45c9-4d78-861b-e37fc5c13af6-0', usage_metadata={'input_tokens': 117, 'output_tokens': 31, 'total_tokens': 148, 'input_token_details': {}, 'output_token_details': {}}), HumanMessage(content='今天遇到了1个小偷', 

In [23]:
response = chat.invoke(messages)
response

AIMessage(content='<think>\n\n</think>\n\n对不起，我还没有学会回答这个问题。如果你有其他问题，我非常乐意为你提供帮助。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 155, 'total_tokens': 180, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-r1:1.5b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-274', 'finish_reason': 'stop', 'logprobs': None}, id='run-0df6a915-bc5a-4aa8-a58b-d37cfc894f27-0', usage_metadata={'input_tokens': 155, 'output_tokens': 25, 'total_tokens': 180, 'input_token_details': {}, 'output_token_details': {}})

In [26]:

prompt = chat_template
output_parser = StrOutputParser()

chain = prompt | chat | output_parser

# llmchain = LLMChain(llm=chat,prompt=chat_template)
# llmchain.run({"name":"咪咪", "user_input":"想我了吗?"})
chain.invoke({"name":"咪咪", "user_input":"想我了吗?"})

'<think>\n\n</think>\n\n对不起，我还没有学会回答这个问题。如果你有其他问题，我非常乐意为你提供帮助。'